In [1]:
#!pip install tensorflow-recommenders

In [2]:
import tensorflow as tf
#import tensorflow_recommenders as tfrs 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
anime_df=pd.read_csv('/content/anime.csv')
rating_df=pd.read_csv('/content/rating.csv')

In [4]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [5]:
anime_df.dtypes

anime_id      int64
name         object
genre        object
type         object
episodes     object
rating      float64
members       int64
dtype: object

In [6]:
rating_df=rating_df[rating_df!=-1.0]
rating_df.dropna()

,user_id,anime_id,rating
47,1,8074,10.0
81,1,11617,10.0
83,1,11757,10.0
101,1,15451,10.0
153,2,11771,10.0
...,...,...,...
1607817,15610,73,8.0
1607818,15610,121,9.0
1607819,15610,136,7.0
1607820,15610,164,9.0


# IBM Collaborative Recomm system


Extract genre column since is not requiered

In [7]:
anime_df.drop('genre',axis=1,inplace=True)

In [8]:
anime_df.head()

,anime_id,name,type,episodes,rating,members
0,32281,Kimi no Na wa.,Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665
2,28977,Gintama°,TV,51,9.25,114262
3,9253,Steins;Gate,TV,24,9.17,673572
4,9969,Gintama&#039;,TV,51,9.16,151266


In [9]:
this_id=anime_df['anime_id'][anime_df['name']=='Dragon Ball']

In [10]:
this_id

346    223
Name: anime_id, dtype: int64

In [11]:
print(anime_df[anime_df.name=='Bleach'])

     anime_id    name type episodes  rating  members
582       269  Bleach   TV      366    7.95   624055


Create User

In [12]:
userInput=(
    {'name':'Dragon Ball','rating':1.0},
    {'name':'One Piece','rating':1.0},
    {'name':'Naruto','rating':1.5},
    {'name':'Death Note','rating':9.0},
    {'name':'Bleach','rating':2.0},
    {'name':'Fullmetal Alchemist: Brotherhood','rating':4.0},
    {'name':'Love Live! School Idol Project','rating':8.0},
    {'name':'White Album 2','rating':9.0},
    {'name':'White Album','rating':8.0},
    {'name':'Fairy Tail','rating':7.0},
    {'name':'Soul Eater','rating':3.0},
    {'name':'Haiyore! Nyaruko-san W','rating':6.0},
    {'name':'Shigatsu wa Kimi no Uso','rating':10.0},
    {'name':'Myself; Yourself','rating':8.0},
    {'name':'True Tears','rating':8.0},
    {'name':'Nanatsu no Taizai','rating':1.0},
    {'name':'Stranger: Mukou Hadan','rating':10.0},
    {'name':'Kimi no Na wa.','rating':9.0},
    {'name':'Digimon','rating':9.0},
    {'name':'Pokemon','rating':8.0}






)
input_anime=pd.DataFrame(userInput)
input_anime

,name,rating
0,Dragon Ball,1.0
1,One Piece,1.0
2,Naruto,1.5
3,Death Note,9.0
4,Bleach,2.0
5,Fullmetal Alchemist: Brotherhood,4.0
6,Love Live! School Idol Project,8.0
7,White Album 2,9.0
8,White Album,8.0
9,Fairy Tail,7.0


In [13]:
input_anime.name

0                          Dragon Ball
1                            One Piece
2                               Naruto
3                           Death Note
4                               Bleach
5     Fullmetal Alchemist: Brotherhood
6       Love Live! School Idol Project
7                        White Album 2
8                          White Album
9                           Fairy Tail
10                          Soul Eater
11              Haiyore! Nyaruko-san W
12             Shigatsu wa Kimi no Uso
13                    Myself; Yourself
14                          True Tears
15                   Nanatsu no Taizai
16               Stranger: Mukou Hadan
17                      Kimi no Na wa.
18                             Digimon
19                             Pokemon
Name: name, dtype: object

Match input user names to the corresponding anime_id

In [14]:
inputId = anime_df[anime_df['name'].isin(input_anime['name'].tolist())]
inputId


,anime_id,name,type,episodes,rating,members
0,32281,Kimi no Na wa.,Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665
16,23273,Shigatsu wa Kimi no Uso,TV,22,8.92,416397
40,1535,Death Note,TV,37,8.71,1013917
74,21,One Piece,TV,Unknown,8.58,504862
136,2418,Stranger: Mukou Hadan,Movie,1,8.43,125503
141,23755,Nanatsu no Taizai,TV,24,8.42,331627
288,6702,Fairy Tail,TV,175,8.22,584590
346,223,Dragon Ball,TV,153,8.16,316102
440,3588,Soul Eater,TV,51,8.08,580184


In [15]:
input_anime=pd.merge(inputId,input_anime,on='name')
input_anime

,anime_id,name,type,episodes,rating_x,members,rating_y
0,32281,Kimi no Na wa.,Movie,1,9.37,200630,9.0
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,4.0
2,23273,Shigatsu wa Kimi no Uso,TV,22,8.92,416397,10.0
3,1535,Death Note,TV,37,8.71,1013917,9.0
4,21,One Piece,TV,Unknown,8.58,504862,1.0
5,2418,Stranger: Mukou Hadan,Movie,1,8.43,125503,10.0
6,23755,Nanatsu no Taizai,TV,24,8.42,331627,1.0
7,6702,Fairy Tail,TV,175,8.22,584590,7.0
8,223,Dragon Ball,TV,153,8.16,316102,1.0
9,3588,Soul Eater,TV,51,8.08,580184,3.0


In [16]:
input_anime.drop(['episodes','rating_x','members','type'],axis=1,inplace=True)
input_anime

,anime_id,name,rating_y
0,32281,Kimi no Na wa.,9.0
1,5114,Fullmetal Alchemist: Brotherhood,4.0
2,23273,Shigatsu wa Kimi no Uso,10.0
3,1535,Death Note,9.0
4,21,One Piece,1.0
5,2418,Stranger: Mukou Hadan,10.0
6,23755,Nanatsu no Taizai,1.0
7,6702,Fairy Tail,7.0
8,223,Dragon Ball,1.0
9,3588,Soul Eater,3.0


In [17]:
input_anime=input_anime.rename(columns={'rating_y':'rating'})
input_anime

,anime_id,name,rating
0,32281,Kimi no Na wa.,9.0
1,5114,Fullmetal Alchemist: Brotherhood,4.0
2,23273,Shigatsu wa Kimi no Uso,10.0
3,1535,Death Note,9.0
4,21,One Piece,1.0
5,2418,Stranger: Mukou Hadan,10.0
6,23755,Nanatsu no Taizai,1.0
7,6702,Fairy Tail,7.0
8,223,Dragon Ball,1.0
9,3588,Soul Eater,3.0


Create set of users with similar ratings

In [18]:
userSubset = rating_df[rating_df['anime_id'].isin(input_anime['anime_id'].tolist())]
userSubset.head()

,user_id,anime_id,rating
0,1,20,NaN
156,3,20,8.0
163,3,527,7.0
173,3,1535,10.0
181,3,3588,8.0


In [19]:
userSubset.shape

(43734, 3)

In [20]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup.get_group(200)

,user_id,anime_id,rating
15534,200,1535,10.0
15541,200,3588,10.0


In [21]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [22]:
userSubsetGroup[0:3]

[(7345,         user_id  anime_id  rating
  792115     7345        20     6.0
  792266     7345       223     7.0
  792300     7345       269     6.0
  792440     7345       527     6.0
  792781     7345      1535    10.0
  792941     7345      2129     7.0
  792992     7345      2418    10.0
  793060     7345      2926     7.0
  793147     7345      3588     5.0
  793265     7345      4720     7.0
  793311     7345      5114     9.0
  793487     7345      6702     7.0
  794139     7345     15051     7.0
  794161     7345     15699     5.0
  794247     7345     18245     5.0
  794395     7345     23273     9.0
  794410     7345     23755     6.0), (8115,         user_id  anime_id  rating
  893739     8115        20     7.0
  893814     8115       223     8.0
  893832     8115       269     7.0
  893887     8115       527     6.0
  893991     8115      1535    10.0
  894038     8115      2129     8.0
  894057     8115      2418     8.0
  894080     8115      2926     5.0
  894119     81

In [23]:


userSubsetGroup = userSubsetGroup[0:1000]



In [24]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='anime_id')
    input_anime = input_anime.sort_values(by='anime_id')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = input_anime[input_anime['anime_id'].isin(group['anime_id'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [25]:
pearsonCorrelationDict.items()

dict_items([(7345, 0.43181083517976615), (8115, -0.06924250663197523), (4468, -0.02730575816362342), (4512, 0.4305875158539973), (7247, -0.44817158552148173), (226, 0.2236238334554357), (1176, -0.16044200853966725), (2378, 0.1571112831034423), (3325, nan), (4174, nan), (5669, -0.2884890267152957), (8149, nan), (10240, 0.0), (11339, nan), (11867, -0.2986904608397578), (12402, -0.1714193110113325), (13894, -0.24031207430288137), (14682, nan), (1504, 0.18050076379674967), (3203, 0.06188853959909328), (3338, nan), (4177, -0.508991157614851), (4249, -0.09659844574351241), (6971, 0.24453921103976367), (9032, -0.09362511734474613), (10140, 0.15659299599812918), (10231, -0.5086075027184003), (10419, 0.25707079367519825), (10785, 0.21547043553506562), (11594, nan), (12138, 0.21609760371464679), (13877, -0.128145656245287), (14764, nan), (14938, nan), (392, 0.017643243655649236), (741, -0.1481176374468417), (813, 0.18739716903415105), (1344, nan), (1349, -0.3213291960680524), (1366, 0.4935949128

In [26]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['user_id'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,user_id
0,0.431811,7345
1,-0.069243,8115
2,-0.027306,4468
3,0.430588,4512
4,-0.448172,7247


In [27]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,user_id
304,0.961769,551
971,0.930783,427
719,0.920032,7325
319,0.899319,1309
368,0.870435,4404


In [28]:
topUsersRating=topUsers.merge(rating_df, left_on='user_id', right_on='user_id', how='inner')
topUsersRating.head()

,similarityIndex,user_id,anime_id,rating
0,0.961769,551,6,7.0
1,0.961769,551,20,6.0
2,0.961769,551,22,10.0
3,0.961769,551,30,8.0
4,0.961769,551,63,5.0


In [29]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,user_id,anime_id,rating,weightedRating
0,0.961769,551,6,7.0,6.732384
1,0.961769,551,20,6.0,5.770615
2,0.961769,551,22,10.0,9.617692
3,0.961769,551,30,8.0,7.694154
4,0.961769,551,63,5.0,4.808846


In [30]:
tempTopUsersRating = topUsersRating.groupby('anime_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
anime_id,,
1,12.264426,107.190556
5,5.063637,43.954274
6,10.724382,85.204646
7,1.961538,13.730768
8,0.633862,4.437036


In [31]:
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['anime_id'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,anime_id
anime_id,,
1,8.739957,1
5,8.680377,5
6,7.944947,6
7,7.000000,7
8,7.000000,8


In [32]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,anime_id
anime_id,,
2164,10.0,2164
7058,10.0,7058
7748,10.0,7748
9735,10.0,9735
17371,10.0,17371
27509,10.0,27509
5244,10.0,5244
25897,10.0,25897
108,10.0,108


In [33]:
anime_df.loc[anime_df['anime_id'].isin(recommendation_df.head(10)['anime_id'].tolist())]

,anime_id,name,type,episodes,rating,members
216,9735,Gintama: Shinyaku Benizakura-hen,Special,1,8.31,14002
313,2164,Dennou Coil,TV,26,8.19,76860
1372,5244,Aria The Natural: Sono Futatabi Deaeru Kiseki ...,Special,1,7.56,10069
1832,108,Ou Dorobou Jing in Seventh Heaven,OVA,3,7.43,8561
1847,7058,Uragiri wa Boku no Namae wo Shitteiru,TV,24,7.43,55040
3007,27509,Toaru Majutsu no Index 10th Anniversary PV,Special,1,7.11,5052
3834,25897,Love Live! School Idol Project: μ&#039;s →NEXT...,Special,3,6.87,5282
4726,11583,Kami nomi zo Shiru Sekai: Natsu-iro Surprise,Music,1,6.65,5772
11138,7748,Sono Hanabira ni Kuchizuke wo: Anata to Koibit...,OVA,1,7.40,24288
11830,17371,Natsuyasumi.,ONA,3,6.02,4388


Optional (Content Base)

In [34]:
anime_df=pd.read_csv('/content/anime.csv')
rating_df=pd.read_csv('/content/rating.csv')

In [35]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [36]:
anime_df['genre'] = anime_df.genre.str.split(', ')
anime_df.drop('rating',axis=1,inplace=True)
anime_df.head()

,anime_id,name,genre,type,episodes,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,151266


In [37]:
anime_withGenre=anime_df.copy()



In [38]:
for index,row in anime_df.iterrows():
  for genre in list(row['genre']):
    anime_withGenre.at[index, genre] = 1.0



  

TypeError: ignored

In [39]:
anime_withGenre = anime_withGenre.fillna(0)
anime_withGenre.head()

,anime_id,name,genre,type,episodes,members,Drama,Romance,School,Supernatural,...,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,793665,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,114262,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,673572,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,151266,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
userInput=(
    {'name':'Dragon Ball','rating':1.0},
    {'name':'One Piece','rating':1.0},
    {'name':'Naruto','rating':1.5},
    {'name':'Death Note','rating':9.0},
    {'name':'Bleach','rating':2.0},
    {'name':'Fullmetal Alchemist: Brotherhood','rating':4.0},
    {'name':'High School DxD','rating':6.0},
    {'name':'Highschool of the Dead','rating':8.0},
    {'name':'Love Live! School Idol Project','rating':8.0},
    {'name':'White Album 2','rating':9.0},
    {'name':'White Album','rating':8.0},
    {'name':'Fairy Tail','rating':7.0},
    {'name':'Soul Eater','rating':3.0},
    {'name':'Haiyore! Nyaruko-san W','rating':6.0},
    {'name':'Shigatsu wa Kimi no Uso','rating':10.0},
    {'name':'Myself; Yourself','rating':8.0},
    {'name':'True Tears','rating':8.0},
    {'name':'Nanatsu no Taizai','rating':1.0},
    {'name':'Stranger: Mukou Hadan','rating':10.0},
    {'name':'Kimi no Na wa.','rating':9.0},





)
input_anime=pd.DataFrame(userInput)
input_anime

,name,rating
0,Dragon Ball,1.0
1,One Piece,1.0
2,Naruto,1.5
3,Death Note,9.0
4,Bleach,2.0
5,Fullmetal Alchemist: Brotherhood,4.0
6,High School DxD,6.0
7,Highschool of the Dead,8.0
8,Love Live! School Idol Project,8.0
9,White Album 2,9.0


In [41]:
inputId = anime_df[anime_df['name'].isin(input_anime['name'].tolist())]
input_anime=pd.merge(inputId,input_anime,on='name')


In [42]:
input_anime.drop(['episodes','members','type'],axis=1,inplace=True)
input_anime

,anime_id,name,genre,rating
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",9.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",4.0
2,23273,Shigatsu wa Kimi no Uso,"[Drama, Music, Romance, School, Shounen]",10.0
3,1535,Death Note,"[Mystery, Police, Psychological, Supernatural,...",9.0
4,21,One Piece,"[Action, Adventure, Comedy, Drama, Fantasy, Sh...",1.0
5,2418,Stranger: Mukou Hadan,"[Action, Adventure, Historical, Samurai]",10.0
6,23755,Nanatsu no Taizai,"[Action, Adventure, Ecchi, Fantasy, Shounen, S...",1.0
7,6702,Fairy Tail,"[Action, Adventure, Comedy, Fantasy, Magic, Sh...",7.0
8,223,Dragon Ball,"[Adventure, Comedy, Fantasy, Martial Arts, Sho...",1.0
9,3588,Soul Eater,"[Action, Adventure, Comedy, Fantasy, Shounen, ...",3.0


In [43]:
#input_anime.drop(['anime_id_y'],axis=1,inplace=True)


In [44]:
input_anime.drop('genre',axis=1,inplace=True)

In [45]:
input_anime.head()

,anime_id,name,rating
0,32281,Kimi no Na wa.,9.0
1,5114,Fullmetal Alchemist: Brotherhood,4.0
2,23273,Shigatsu wa Kimi no Uso,10.0
3,1535,Death Note,9.0
4,21,One Piece,1.0


In [46]:
userAnime = anime_withGenre[anime_withGenre['anime_id'].isin(input_anime['anime_id'].tolist())]
userAnime

,anime_id,name,genre,type,episodes,members,Drama,Romance,School,Supernatural,...,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,793665,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,23273,Shigatsu wa Kimi no Uso,"[Drama, Music, Romance, School, Shounen]",TV,22,416397,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1535,Death Note,"[Mystery, Police, Psychological, Supernatural,...",TV,37,1013917,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,21,One Piece,"[Action, Adventure, Comedy, Drama, Fantasy, Sh...",TV,Unknown,504862,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,2418,Stranger: Mukou Hadan,"[Action, Adventure, Historical, Samurai]",Movie,1,125503,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,23755,Nanatsu no Taizai,"[Action, Adventure, Ecchi, Fantasy, Shounen, S...",TV,24,331627,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,6702,Fairy Tail,"[Action, Adventure, Comedy, Fantasy, Magic, Sh...",TV,175,584590,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
346,223,Dragon Ball,"[Adventure, Comedy, Fantasy, Martial Arts, Sho...",TV,153,316102,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,3588,Soul Eater,"[Action, Adventure, Comedy, Fantasy, Shounen, ...",TV,51,580184,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
userAnime = userAnime.reset_index(drop=True)

In [48]:
userGenreTable = userAnime.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('type', 1).drop('members',1).drop('episodes',1)
userGenreTable

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,...,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
#userGenreTable.drop('rating',1)

In [50]:


input_anime['rating']



0      9.0
1      4.0
2     10.0
3      9.0
4      1.0
5     10.0
6      1.0
7      7.0
8      1.0
9      3.0
10     9.0
11     2.0
12     1.5
13     6.0
14     8.0
15     8.0
16     8.0
17     6.0
18     8.0
19     8.0
Name: rating, dtype: float64

In [51]:
userProfile = userGenreTable.transpose().dot(input_anime['rating'])
#The user profile
userProfile

Drama            49.0
Romance          50.0
School           49.0
Supernatural     32.0
Action           37.5
Adventure        27.0
Fantasy          17.0
Magic            11.0
Military          4.0
Shounen          30.5
Comedy           27.5
Historical       10.0
Parody            6.0
Samurai          10.0
Sci-Fi            6.0
Thriller          9.0
Sports            0.0
Super Power       5.5
Space             0.0
Slice of Life    17.0
Mecha             0.0
Music            27.0
Mystery           9.0
Seinen            0.0
Martial Arts      2.5
Vampire           0.0
Shoujo            0.0
Horror            8.0
Police            9.0
Psychological     9.0
Demons            6.0
Ecchi            15.0
Josei             0.0
Shounen Ai        0.0
Game              0.0
Dementia          0.0
Harem             6.0
Cars              0.0
Kids              0.0
Shoujo Ai         0.0
dtype: float64

In [52]:
genreTable = anime_withGenre.set_index(anime_withGenre['anime_id'])
#And drop the unnecessary information
genreTable = genreTable.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('type', 1)
genreTable.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,episodes,members,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,...,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,1,200630,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,64,793665,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,51,114262,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,24,673572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,51,151266,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
genreTable.drop(['episodes','members'],axis=1,inplace=True)

In [54]:
genreTable.shape

(12294, 40)

In [55]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

anime_id
32281    0.367722
5114     0.359551
28977    0.260470
9253     0.030644
9969     0.260470
dtype: float64

In [56]:


#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()



anime_id
25157    0.562819
4938     0.537283
1397     0.528090
969      0.514811
451      0.508682
dtype: float64

In [57]:
anime_df.loc[anime_df['anime_id'].isin(recommendationTable_df.head(30).keys())]

,anime_id,name,genre,type,episodes,members
151,33,Berserk,"[Action, Adventure, Demons, Drama, Fantasy, Ho...",TV,25,226430
173,6811,InuYasha: Kanketsu-hen,"[Action, Adventure, Comedy, Demons, Fantasy, M...",TV,26,99128
180,18195,Little Busters!: Refrain,"[Comedy, Drama, Romance, School, Slice of Life...",TV,13,71820
223,2167,Clannad,"[Comedy, Drama, Romance, School, Slice of Life...",TV,23,566690
286,4938,Tsubasa: Shunraiki,"[Action, Adventure, Drama, Fantasy, Magic, Mys...",OVA,2,40420
320,16001,Kokoro Connect: Michi Random,"[Comedy, Drama, Romance, School, Slice of Life...",Special,4,106989
322,232,Cardcaptor Sakura,"[Adventure, Comedy, Drama, Fantasy, Magic, Rom...",TV,70,181249
518,11887,Kokoro Connect,"[Comedy, Drama, Romance, School, Slice of Life...",TV,13,283847
613,20517,Little Busters!: EX,"[Comedy, Drama, Romance, School, Slice of Life...",Special,8,26709
690,249,InuYasha,"[Action, Adventure, Comedy, Demons, Fantasy, M...",TV,167,281632


In [58]:
#new=anime_df.genre.str.split(', ',expand=True)

In [59]:
#df=pd.merge(rating_df,anime_data,on='anime_id')